In [ ]:
import numpy as np
import os
import random
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, Lambda
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras import backend as K
from tensorflow.keras.layers import BatchNormalization, Dropout

def create_siamese_network(input_shape):
    image_input = Input(shape=input_shape)

    conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
    conv1 = BatchNormalization()(conv1)

    conv2 = Conv2D(64, (3, 3), activation='relu')(conv1)
    conv2 = BatchNormalization()(conv2)

    conv3 = Conv2D(128, (3, 3), activation='relu')(conv2)
    conv3 = BatchNormalization()(conv3)

    conv4 = Conv2D(128, (3, 3), activation='relu')(conv3)
    conv4 = BatchNormalization()(conv4)
    flatten = Flatten()(conv4)

    fc1 = Dense(256, activation='relu')(flatten)
    fc1 = Dropout(0.2)(fc1)

    fc2 = Dense(128, activation='relu')(fc1)
    fc2 = Dropout(0.1)(fc2)

    fc3 = Dense(64, activation='relu')(fc2)

    model = Model(inputs=image_input, outputs=fc3)

    return model

def contrastive_loss(y_true, y_pred):
    margin = 1
    y_true = K.cast(y_true, dtype=tf.float32)  # Cast y_true to float32
    return K.mean(
        y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0))
    )


def euclidean_distance(vectors):
    x, y = vectors
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def create_siamese_network_model(input_shape):
    siamese_network = create_siamese_network(input_shape)
    input_image_1 = Input(shape=input_shape)
    input_image_2 = Input(shape=input_shape)
    output_1 = siamese_network(input_image_1)
    output_2 = siamese_network(input_image_2)

    distance = Lambda(euclidean_distance)([output_1, output_2])

    model = Model(inputs=[input_image_1, input_image_2], outputs=distance)

    return model

# Set the input shape of the images
input_shape = (64, 64, 3)

# Create the Siamese network model
siamese_model = create_siamese_network_model(input_shape)

# Compile the model
siamese_model.compile(optimizer='adam', loss=contrastive_loss)

# Paths to directories containing similar and dissimilar image pairs
similar_image_dir = '/content/drive/MyDrive/Siamese_Dataset/similar'
dissimilar_image_dir = '/content/drive/MyDrive/Siamese_Dataset/dissimilar'

# Get the list of filenames in each directory
similar_image_files = [f for f in os.listdir(similar_image_dir) if os.path.isfile(os.path.join(similar_image_dir, f))]
dissimilar_image_files = [f for f in os.listdir(dissimilar_image_dir) if os.path.isfile(os.path.join(dissimilar_image_dir, f))]

# Shuffle the lists of filenames
random.shuffle(similar_image_files)
random.shuffle(dissimilar_image_files)

# Define the number of similar and dissimilar pairs to use for training
num_pairs = min(len(similar_image_files), len(dissimilar_image_files))
num_training_pairs = min(int(0.8 * num_pairs), len(similar_image_files) - 1, len(dissimilar_image_files) - 1)

# Initialize the lists for training image pairs and labels
training_image_pairs = []
training_labels = []

# Generate training data with similar image pairs
for i in range(num_training_pairs):
    # Select a random pair of similar images
    similar_image_1 = similar_image_files[i]
    similar_image_2 = similar_image_files[i + 1]

    # Add the image pair and label to the training lists
    training_image_pairs.append([
        os.path.join(similar_image_dir, similar_image_1),
        os.path.join(similar_image_dir, similar_image_2)
    ])
    training_labels.append(1)  # Label as similar

# Generate training data with dissimilar image pairs
for i in range(num_training_pairs):
    # Select a random pair of dissimilar images
    dissimilar_image_1 = dissimilar_image_files[i]
    dissimilar_image_2 = dissimilar_image_files[i + 1]

    # Add the image pair and label to the training lists
    training_image_pairs.append([
        os.path.join(dissimilar_image_dir, dissimilar_image_1),
        os.path.join(dissimilar_image_dir, dissimilar_image_2)
    ])
    training_labels.append(0)  # Label as dissimilar

# Convert the training image pairs and labels to numpy arrays
training_image_pairs = np.array(training_image_pairs)
training_labels = np.array(training_labels)

# Fit the model on the training data
siamese_model.fit(
    [np.array([img_to_array(load_img(pair[0], target_size=input_shape[:2])) / 255.0 for pair in training_image_pairs if os.path.isfile(pair[0])]),
     np.array([img_to_array(load_img(pair[1], target_size=input_shape[:2])) / 255.0 for pair in training_image_pairs if os.path.isfile(pair[1])])],
    training_labels,
    epochs=10,
    batch_size=32
)



# Save the model
siamese_model.save('siamese_model.h5')


Epoch 1/10
1/1 [==============================] - 12s 12s/step - loss: 16.9431
Epoch 2/10
1/1 [==============================] - 9s 9s/step - loss: 110613.2734
Epoch 3/10
1/1 [==============================] - 7s 7s/step - loss: 82735.5312
Epoch 4/10
1/1 [==============================] - 8s 8s/step - loss: 24420.7480
Epoch 5/10
1/1 [==============================] - 7s 7s/step - loss: 22875.4785
Epoch 6/10
1/1 [==============================] - 8s 8s/step - loss: 19745.2969
Epoch 7/10
1/1 [==============================] - 8s 8s/step - loss: 21122.9902
Epoch 8/10
1/1 [==============================] - 8s 8s/step - loss: 8278.6621
Epoch 9/10
1/1 [==============================] - 9s 9s/step - loss: 28387.9316
Epoch 10/10
1/1 [==============================] - 7s 7s/step - loss: 18986.7949


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def contrastive_loss(y_true, y_pred):
    margin = 1
    y_true = K.cast(y_true, dtype=tf.float32)  # Cast y_true to float32
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

# Define the custom_objects dictionary with the contrastive_loss function
custom_objects = {'contrastive_loss': contrastive_loss}

# Load the saved model with the custom loss function
siamese_model = load_model('siamese_model.h5', custom_objects=custom_objects)

# Set the input shape of the images
input_shape = (64, 64, 3)

# Load and preprocess two image files
image_file_1 = '/content/drive/MyDrive/Siamese_Dataset/similar/edsheeran_cover.jpg'
image_file_2 = '/content/drive/MyDrive/Siamese_Dataset/dissimilar/badCover.jpg'
image_1 = img_to_array(load_img(image_file_1, target_size=input_shape[:2])) / 255.0
image_2 = img_to_array(load_img(image_file_2, target_size=input_shape[:2])) / 255.0
image_1 = np.expand_dims(image_1, axis=0)
image_2 = np.expand_dims(image_2, axis=0)

# Predict the distance between the two images
distance = siamese_model.predict([image_1, image_2])
print('Distance:', distance)


1/1 [==============================] - 0s 413ms/step
Distance: [[2.696117]]
